### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

# from dotenv import load_dotenv

# load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons in favor of choosing AutoGen for your AI Agent project:

1. **Customizable Roles**: AutoGen allows you to tailor agents to fit specific jobs, enabling a high degree of customization in their roles and capabilities. This means you can easily adapt agents to meet the unique requirements of your project.

2. **Multi-Agent Collaboration**: The framework supports multiple AI agents that can communicate and collaborate. This enhances problem-solving abilities by allowing agents to share information and refine results collectively, thus improving efficiency.

3. **Enhanced Efficiency**: AutoGen streamlines complex tasks and reduces the need for manual intervention, leading to faster execution and better resource management. 

4. **Support for Complex Tasks**: By supporting various tools, like code executors and function callers, AutoGen enables agents to handle intricate and iterative problem-solving, making it ideal for open-ended tasks.

5. **Lowered Entry Barrier**: AutoGen provides a more accessible framework for developers, allowing even those without extensive experience in AI to create and deploy intelligent agents effectively.

6. **Enriched Conversations**: The system excels in facilitating multi-agent discussions, which enhances the interaction and inference capabilities of Large Language Models (LLMs), leading to more robust and intelligent solutions.

Overall, AutoGen promotes a scalable, efficient, and collaborative approach to developing AI agents, making it a strong choice for your project.

TERMINATE.

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI Agent project:

1. **Complexity**: AutoGen can present considerable complexity, particularly for newcomers, requiring a steep learning curve that may hinder initial development speed and efficiency.

2. **Inaccurate Handling of Complex Queries**: It struggles with multihop questions that require synthesizing information from multiple sources, leading to potential inaccuracies.

3. **Limitations in Creativity**: While AutoGen can generate content based on predefined inputs, it lacks the creativity and flexibility needed for more nuanced or innovative tasks.

4. **Dependence on Templates**: Its reliance on predefined templates can stifle adaptability to unique or unexpected scenarios, limiting its overall effectiveness in dynamic environments.

5. **High Initial Costs**: Implementation can be costly, making it a less attractive option for budget-conscious projects.

6. **Job Displacement Concerns**: The use of AutoGen raises ethical concerns regarding job displacement, as it may reduce the need for human labor in certain tasks.

7. **Security and Privacy Issues**: Like many AI systems, AutoGen might raise security and privacy concerns, particularly in how it processes and stores data.

These drawbacks should be carefully considered in the context of your project's requirements and objectives. 

TERMINATE



## Decision:

Based on the provided research, I would recommend using AutoGen for the project. 

The pros highlight its strong capabilities in customization, multi-agent collaboration, and enhanced efficiency which are crucial for developing effective AI agents. The advantages in handling complex tasks and facilitating enriched conversations can lead to better outcomes in problem-solving. Despite the cons regarding complexity and initial costs, the overall benefits, especially the accessibility for developers and collaborative features, make AutoGen a valuable asset for achieving project goals effectively.

TERMINATE.

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()